In [1]:
import utils

import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_similarity_score

import keras
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, InputLayer
from keras.optimizers import RMSprop, Adam, SGD

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import scipy

from os.path import join, basename, exists
from os import makedirs, listdir

/root/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [46]:
def preproc_target_train(data, reduce_classes=False):
    diagnoses = data['Код_диагноза'].copy()
    
    if reduce_classes:
        pop_diagnoses = set(utils.get_most_popular_diagnoses(diagnoses, percent=.80))
        most_pop_diagnose = scipy.stats.mode(diagnoses)[0][0]
    else:
        pop_diagnoses = set(diagnoses)
        most_pop_diagnose = scipy.stats.mode(diagnoses)[0][0]
    
    diagnoses = diagnoses.apply(
        lambda diag: diag if diag in pop_diagnoses else most_pop_diagnose
    )
    
    return diagnoses, pop_diagnoses, most_pop_diagnose

def preproc_target_test(data, pop_diagnoses, most_pop_diagnose):
    diagnoses = data['Код_диагноза'].copy()
    
    diagnoses = diagnoses.apply(
        lambda diag: diag if diag in pop_diagnoses else most_pop_diagnose
    )
    
    return diagnoses, pop_diagnoses, most_pop_diagnose

def join_topics(data, topics):
    data = data.copy()
    
    topics_df = pd.DataFrame(dict(zip(
        ['topic' + str(i) for i in range(topics.shape[1])],
        [topics[:, i] for i in range(topics.shape[1])])))
    topics_df['Id_Записи'] = data['Id_Записи']

    data = data.join(topics_df, on='Id_Записи', rsuffix='_topics', how='outer')
    data = data.drop(columns=['Id_Записи_topics'])
    
    return data

In [44]:
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

class DoctorsPopularityTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        doctors = x.fillna('sss')
        doctors_voc, counts = np.unique(doctors, return_counts=True)
        self.pop_doctor = doctors_voc[np.argsort(counts)[::-1][0]]
        
        return self

    def transform(self, x):
        x = x.fillna('sss')
        x[x == 'sss'] = self.pop_doctor
        
        return x
    
class GenderTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        x = x.copy()
        x[x == 1] = 0
        x[x == 2] = 1
        
        return np.expand_dims(x, axis=1)
    
class AgeTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        return np.expand_dims(x, axis=1)
    
class TopicsTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        return x

In [33]:
class NNModel(BaseEstimator, TransformerMixin):
    def __init__(self, batch_size, epochs, tb_log_dir, model_path, validation_split=0.3):
        self.batch_size = batch_size
        self.epochs = epochs
        self.tb_log_dir = tb_log_dir
        self.model_path = model_path
        self.validation_split = validation_split
    
    def fit(self, x, y=None):
        self.classes = np.unique(y)
        self.classes_voc = dict(zip(self.classes, range(self.classes.shape[0])))
        self.voc_classes = dict(zip(range(self.classes.shape[0]), self.classes))
        y_proc = np.zeros((y.shape[0], self.classes.shape[0]), dtype=np.float32)
        for i, yc in enumerate(y):
            y_proc[i, self.classes_voc[yc]] = 1.
        
        self.model = Sequential([
            InputLayer(input_shape=(x.shape[1],)),
            Dense(2048, activation='sigmoid'),
            Dense(2048, activation='sigmoid'),
            Dense(self.classes.shape[0], activation='softmax')
        ])
        
        optim = RMSprop(lr=1e-4, decay=1e-6)
        self.model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy'])
        
        self.model.fit(x, y_proc,
                  batch_size=self.batch_size, epochs=self.epochs,
                  callbacks=[
                      TensorBoard(log_dir=self.tb_log_dir, batch_size=self.batch_size),
                      ModelCheckpoint(filepath=self.model_path, monitor='acc', period=5)
                  ],
                  validation_split=self.validation_split)
        
        return self
        
    def transform(self, x):
        pred = self.model.predict(x)
        max_idxs = np.argmax(pred, axis=1)
        
        return np.array(list(map(lambda max_idx: self.voc_classes[max_idx], max_idxs)))

In [45]:
no_validation = False
validation_split = 0.3

experiment_dir = 'simple_models'
model_name = 'nn_3dense' + str(utils.get_next_model_id(experiment_dir))
tb_log_dir = join(experiment_dir, 'log', model_name)
models_dir = join(experiment_dir, 'models')

if no_validation:
    model_path = utils.get_model_fname_pattern(models_dir, model_name, no_validation=True)
    validation_split = 0.0
else:
    model_path = utils.get_model_fname_pattern(models_dir, model_name, no_validation=False)

pipe = Pipeline([
    ('union', FeatureUnion(
        transformer_list = [
            ('complaints_pipe', Pipeline([
                ('complaint_selector', ItemSelector(key='Жалобы (unigramm)')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,1), min_df=10, stop_words=stopwords.words('russian')))
            ])),
            ('complaints_n_pipe', Pipeline([
                ('complaint_n_selector', ItemSelector(key='Жалобы (ngramm)')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,1), min_df=1, stop_words=stopwords.words('russian')))
            ])),
            ('doctor_pipe', Pipeline([
                ('doctor_selector', ItemSelector(key='Врач')),
                ('doc_pop', DoctorsPopularityTransformator()),
                ('count_vect', CountVectorizer())
            ])),
            ('gender_pipe', Pipeline([
                ('gender_selector', ItemSelector(key='Пол')),
                ('gender_transform', GenderTransformator())
            ])),
            ('age_pipe', Pipeline([
                ('age_selector', ItemSelector(key='Возраст')),
                ('age_transformator', AgeTransformator())
            ])),
            ('topics_pipe', Pipeline([
                ('topic_selector', ItemSelector(key=['topic' + str(i) for i in range(355)])),
                ('topics_transform', TopicsTransformator())
            ]))
        ]
    )),
    ('clf', NNModel(batch_size=128, epochs=200, tb_log_dir=tb_log_dir,
                    model_path=model_path, validation_split=validation_split))
])

In [47]:
train = utils.load_data('data/train_data_complaints_repeats_doctors.csv')
train_topics = np.load('data/topics_train_ngramm.npy')
train = join_topics(train, train_topics)
# train, valid = train_test_split(train, test_size=0.3)

In [48]:
train_y, pop_diagnoses, most_pop_diagnose = preproc_target_train(train, reduce_classes=False)
# valid_y, _, _ = preproc_target_test(valid, pop_diagnoses, most_pop_diagnose)

/root/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [49]:
np.unique(train_y).shape

(2302,)

In [50]:
pipe.fit(train, train_y)

Train on 43383 samples, validate on 18593 samples
Epoch 1/200
43383/43383 [==============================] - 8s 177us/step - loss: 5.6862 - acc: 0.0486 - val_loss: 6.1860 - val_acc: 0.0457
Epoch 2/200
43383/43383 [==============================] - 8s 176us/step - loss: 5.4604 - acc: 0.0716 - val_loss: 6.0422 - val_acc: 0.0769
Epoch 3/200
43383/43383 [==============================] - 8s 179us/step - loss: 5.2197 - acc: 0.0949 - val_loss: 5.8356 - val_acc: 0.0863
Epoch 4/200
43383/43383 [==============================] - 8s 180us/step - loss: 4.9356 - acc: 0.1150 - val_loss: 5.6096 - val_acc: 0.1228
Epoch 5/200
43383/43383 [==============================] - 8s 180us/step - loss: 4.7070 - acc: 0.1615 - val_loss: 5.4025 - val_acc: 0.1518
Epoch 6/200
43383/43383 [==============================] - 8s 177us/step - loss: 4.4800 - acc: 0.1818 - val_loss: 5.2101 - val_acc: 0.1676
Epoch 7/200
43383/43383 [==============================] - 8s 177us/step - loss: 4.2659 - acc: 0.1987 - val_loss: 5.

43383/43383 [==============================] - 8s 178us/step - loss: 2.8231 - acc: 0.3356 - val_loss: 3.9413 - val_acc: 0.2859
Epoch 60/200
43383/43383 [==============================] - 8s 177us/step - loss: 2.8191 - acc: 0.3356 - val_loss: 3.9315 - val_acc: 0.2850
Epoch 61/200
43383/43383 [==============================] - 8s 181us/step - loss: 2.8154 - acc: 0.3373 - val_loss: 3.9596 - val_acc: 0.2876
Epoch 62/200
43383/43383 [==============================] - 8s 175us/step - loss: 2.8099 - acc: 0.3367 - val_loss: 3.9271 - val_acc: 0.2874
Epoch 63/200
43383/43383 [==============================] - 8s 179us/step - loss: 2.8017 - acc: 0.3378 - val_loss: 3.9443 - val_acc: 0.2869
Epoch 64/200
43383/43383 [==============================] - 8s 175us/step - loss: 2.7981 - acc: 0.3385 - val_loss: 3.9692 - val_acc: 0.2882
Epoch 65/200
43383/43383 [==============================] - 8s 177us/step - loss: 2.7962 - acc: 0.3392 - val_loss: 3.9518 - val_acc: 0.2872
Epoch 66/200
43383/43383 [=======

43383/43383 [==============================] - 8s 175us/step - loss: 2.6132 - acc: 0.3682 - val_loss: 3.9657 - val_acc: 0.3022
Epoch 118/200
43383/43383 [==============================] - 8s 177us/step - loss: 2.6171 - acc: 0.3693 - val_loss: 3.9990 - val_acc: 0.3032
Epoch 119/200
43383/43383 [==============================] - 8s 180us/step - loss: 2.6224 - acc: 0.3705 - val_loss: 3.9410 - val_acc: 0.3027
Epoch 120/200
43383/43383 [==============================] - 8s 176us/step - loss: 2.6085 - acc: 0.3710 - val_loss: 3.9514 - val_acc: 0.3019
Epoch 121/200
43383/43383 [==============================] - 8s 178us/step - loss: 2.6030 - acc: 0.3715 - val_loss: 3.9468 - val_acc: 0.3023
Epoch 122/200
43383/43383 [==============================] - 8s 175us/step - loss: 2.6050 - acc: 0.3740 - val_loss: 3.9831 - val_acc: 0.3022
Epoch 123/200
43383/43383 [==============================] - 8s 177us/step - loss: 2.6097 - acc: 0.3723 - val_loss: 4.0050 - val_acc: 0.3022
Epoch 124/200
43383/43383 [

KeyboardInterrupt: 

In [18]:
pred = pipe.transform(train)

In [19]:
np.unique(pred, return_counts=True)

(array(['A46', 'A63.0', 'B00', 'B00.1', 'B00.8', 'B02', 'B07', 'B08.1',
        'B35.1', 'B35.3', 'B35.6', 'B36.0', 'B37', 'B37.3+', 'B97.1',
        'B97.7', 'D17.0', 'D17.1', 'D17.2', 'D18.0', 'D23', 'D23.3',
        'D23.5', 'D23.9', 'D24', 'D25', 'D50', 'D50.9', 'D89.8', 'E01.1',
        'E01.8', 'E03.8', 'E03.9', 'E04.1', 'E04.2', 'E05.0', 'E06.3',
        'E11', 'E22.1', 'E28', 'E28.1', 'E28.8', 'E28.9', 'E66.0', 'E89.0',
        'F41.0', 'F41.2', 'F45.3', 'F95.0', 'G24', 'G24.9', 'G43.0',
        'G44.2', 'G50.0', 'G51.0', 'G55*', 'G55.1*', 'G55.3*', 'G56.0',
        'G56.2', 'G90', 'G90.8', 'G90.9', 'G93.4', 'H00', 'H00.0', 'H00.1',
        'H01.0', 'H04.1', 'H10.0', 'H10.1', 'H10.2', 'H10.3', 'H10.5',
        'H10.8', 'H11.3', 'H15.1', 'H16.1', 'H16.2', 'H20.0', 'H35.0',
        'H35.3', 'H35.4', 'H40.1', 'H52.0', 'H52.1', 'H52.2', 'H52.4',
        'H52.5', 'H60', 'H61.2', 'H65', 'H65.0', 'H65.1', 'H66.0', 'H66.1',
        'H68.0', 'H68.1', 'H81.1', 'H81.2', 'H90.3', 'H93.1', 

## Submit

In [9]:
train = utils.load_data('data/train_data_complaints_repeats_doctors.csv')
train_topics = np.load('data/topics_train_ngramm.npy')
train = join_topics(train, train_topics)
train_y, pop_diagnoses, most_pop_diagnose = preproc_target_train(train, reduce_classes=False)

/root/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [10]:
pipe.fit(train, train_y)

Epoch 1/100
61976/61976 [==============================] - 13s 215us/step - loss: 5.8733 - acc: 0.0998
Epoch 2/100
61976/61976 [==============================] - 13s 212us/step - loss: 5.5176 - acc: 0.1808
Epoch 3/100
61976/61976 [==============================] - 13s 211us/step - loss: 5.1080 - acc: 0.2237
Epoch 4/100
61976/61976 [==============================] - 13s 212us/step - loss: 4.7338 - acc: 0.2444
Epoch 5/100
61976/61976 [==============================] - 13s 211us/step - loss: 4.4498 - acc: 0.2577
Epoch 6/100
61976/61976 [==============================] - 13s 211us/step - loss: 4.2481 - acc: 0.2679
Epoch 7/100
61976/61976 [==============================] - 13s 209us/step - loss: 4.1090 - acc: 0.2773
Epoch 8/100
61976/61976 [==============================] - 13s 209us/step - loss: 4.0043 - acc: 0.2860
Epoch 9/100
61976/61976 [==============================] - 13s 209us/step - loss: 3.9136 - acc: 0.2952
Epoch 10/100
61976/61976 [==============================] - 13s 211us/ste

61976/61976 [==============================] - 13s 207us/step - loss: 3.1456 - acc: 0.3923
Epoch 80/100
61976/61976 [==============================] - 13s 208us/step - loss: 3.1393 - acc: 0.3923
Epoch 81/100
61976/61976 [==============================] - 13s 210us/step - loss: 3.1382 - acc: 0.3930
Epoch 82/100
61976/61976 [==============================] - 13s 207us/step - loss: 3.1367 - acc: 0.3942
Epoch 83/100
61976/61976 [==============================] - 13s 208us/step - loss: 3.1333 - acc: 0.3947
Epoch 84/100
61976/61976 [==============================] - 13s 209us/step - loss: 3.1251 - acc: 0.3940
Epoch 85/100
61976/61976 [==============================] - 13s 207us/step - loss: 3.1240 - acc: 0.3963
Epoch 86/100
61976/61976 [==============================] - 13s 209us/step - loss: 3.1194 - acc: 0.3956
Epoch 87/100
61976/61976 [==============================] - 13s 209us/step - loss: 3.1168 - acc: 0.3966
Epoch 88/100
61976/61976 [==============================] - 13s 208us/step - 

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('complaints_pipe', Pipeline(memory=None,
     steps=[('complaint_selector', ItemSelector(key='Жалобы (unigramm)')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, e...:48_2018/{epoch:02d}_{acc:.2f}.h5',
    tb_log_dir='simple_models/log/nn14', validation_split=0.0))])

In [51]:
test = utils.load_data('data/test_data_complaints_repeats_doctors.csv')
test_topics = np.load('data/topics_test_ngramm.npy')
test = join_topics(test, test_topics)

In [52]:
test_pred = pipe.transform(test)

In [53]:
submit = pd.DataFrame({'Id_Записи': test['Id_Записи'], 'Код_диагноза': test_pred})

In [54]:
submit['Код_диагноза'].value_counts()

M42.1     3327
J06.9     1899
N76.0     1822
Z32.1     1646
N41.1     1034
N77.1*     978
I11        942
M65        789
Z00.0      779
J35.0      609
J00        608
Z01.8      603
K29.9      577
K30        566
D23.9      519
H52.1      462
N60.1      442
M54.2      435
J01.0      424
M54.8      405
G90        384
L30.8      382
K29.5      334
N76.1      296
E03.8      268
I83.9      266
H61.2      249
N30.0      240
D25        236
E04.2      225
          ... 
Z00.1        4
J37.0        4
S83.5        3
E03.9        3
H68.1        3
B35.3        3
T15.1        3
M17          2
M43.9        2
L30.9        2
Z01          2
H66.0        2
G93.4        2
N47          2
M81.0        2
M53.1        2
H65.1        1
J18.0        1
M17.0        1
I83          1
N64.4        1
Z00          1
N10          1
H15.1        1
M47.8        1
H35.4        1
D89.8        1
M06.0        1
J35.3        1
Z35.8        1
Name: Код_диагноза, Length: 212, dtype: int64

In [55]:
submit.to_csv('submit/bow_simple_nn20_3dense_diag_all_compl_uni-n_gram_doctor_topics.csv', header=True, index=False)